In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split

# Step 1: Define a small SMS dataset manually
data = [
    ("Congratulations! You've won a free ticket to Bahamas. Call now!", 1),  # Spam
    ("Your package is out for delivery.", 0),                                # Ham
    ("Win a $1000 Walmart gift card now. Text WIN to 12345.", 1),            # Spam
    ("Don't forget the meeting at 3 PM today.", 0),                         # Ham
    ("Claim your free Amazon gift card by clicking this link.", 1),         # Spam
    ("Let's have lunch tomorrow.", 0),                                      # Ham
    ("Important: Your account statement is available online.", 0),          # Ham
    ("Free entry in a weekly contest. Text WIN to enter now.", 1),          # Spam
    ("I'll call you back in 10 minutes.", 0),                               # Ham
]

# Split the dataset into messages and labels
texts, labels = zip(*data)
texts = list(texts)
labels = list(labels)

# Split into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

# Step 2: Preprocess the dataset
vocab_size = 1000
max_len = 20
tokenizer = tf.keras.layers.TextVectorization(max_tokens=vocab_size, output_sequence_length=max_len)
train_texts_dataset = tf.data.Dataset.from_tensor_slices(train_texts)
tokenizer.adapt(train_texts_dataset.batch(2))

def preprocess_text(texts, labels):
    texts = tokenizer(texts)
    return texts, labels

train_dataset = tf.data.Dataset.from_tensor_slices((train_texts, train_labels))
train_dataset = train_dataset.map(preprocess_text).batch(2).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_texts, test_labels))
test_dataset = test_dataset.map(preprocess_text).batch(2).prefetch(tf.data.AUTOTUNE)

# Step 3: Define the Transformer-based model
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = models.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TransformerTextClassifier(models.Model):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_dim, num_classes, max_len, rate=0.1):
        super().__init__()
        self.embedding = layers.Embedding(vocab_size, embed_dim)
        self.positional_encoding = self.add_weight(
            "positional_encoding",
            shape=(1, max_len, embed_dim),
            initializer="zeros",
            trainable=True
        )
        self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim, rate)
        self.pooling = layers.GlobalAveragePooling1D()
        self.dropout = layers.Dropout(rate)
        self.fc = layers.Dense(num_classes, activation="softmax")

    def call(self, inputs, training=False):
        x = self.embedding(inputs) + self.positional_encoding
        x = self.transformer_block(x, training)
        x = self.pooling(x)
        x = self.dropout(x, training)
        return self.fc(x)

# Initialize the model
embed_dim = 32
num_heads = 2
ff_dim = 64
num_classes = 2

model = TransformerTextClassifier(vocab_size, embed_dim, num_heads, ff_dim, num_classes, max_len)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Step 4: Train the model
model.fit(train_dataset, validation_data=test_dataset, epochs=5)

# Step 5: Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Step 6: Test for


Epoch 1/5
4/4 [==============================] - 3s 115ms/step - loss: 0.8053 - accuracy: 0.5714 - val_loss: 0.5732 - val_accuracy: 0.5000
Epoch 2/5
4/4 [==============================] - 0s 33ms/step - loss: 0.5430 - accuracy: 0.7143 - val_loss: 0.4713 - val_accuracy: 1.0000
Epoch 3/5
4/4 [==============================] - 0s 33ms/step - loss: 0.3504 - accuracy: 1.0000 - val_loss: 0.4869 - val_accuracy: 0.5000
Epoch 4/5
4/4 [==============================] - 0s 33ms/step - loss: 0.2237 - accuracy: 1.0000 - val_loss: 0.2940 - val_accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 83ms/step - loss: 0.2401 - accuracy: 1.0000
Test Loss: 0.2401, Test Accuracy: 1.0000


In [6]:
import numpy as np

# Step 1: Preprocess a single input
def predict_single_input(model, tokenizer, text, max_len):
    # Tokenize and pad the text
    tokenized_text = tokenizer([text])
    padded_text = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_text, maxlen=max_len, padding="post"
    )
    # Make prediction
    prediction = model.predict(padded_text)
    predicted_class = np.argmax(prediction, axis=1)[0]
    confidence = prediction[0][predicted_class]
    return predicted_class, confidence

# Example usage
input_text = "The movie was fantastic! I really enjoyed it."
predicted_class, confidence = predict_single_input(model, tokenizer, input_text, max_len)

class_names = ["Negative", "Positive"]
print(f"Input Text: {input_text}")
print(f"Predicted Class: {class_names[predicted_class]} with confidence {confidence:.2f}")


1/1 [==============================] - 0s 167ms/step
Input Text: The movie was fantastic! I really enjoyed it.
Predicted Class: Positive with confidence 0.56
